In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
!pip install -q -U keras-tuner

In [3]:
import keras_tuner as kt

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

2022-03-01 13:22:41.978518: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [9]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 58s]
val_accuracy: 0.887583315372467

Best val_accuracy So Far: 0.887583315372467
Total elapsed time: 00h 10m 32s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 352 and the optimal learning rate for the optimizer
is 0.001.



In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4988 - accuracy: 0.8247 - val_loss: 0.4043 - val_accuracy: 0.8558
Epoch 2/50
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3730 - accuracy: 0.8650 - val_loss: 0.4027 - val_accuracy: 0.8526
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3312 - accuracy: 0.8778 - val_loss: 0.3532 - val_accuracy: 0.8723
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3071 - accuracy: 0.8862 - val_loss: 0.3505 - val_accuracy: 0.8754
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2866 - accuracy: 0.8939 - val_loss: 0.3170 - val_accuracy: 0.8847
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2711 - accuracy: 0.8996 - val_loss: 0.3135 - val_accuracy: 0.8888
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2579 - accuracy: 0.9025 - val_loss: 0.3107 - val_accuracy:

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/39
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4982 - accuracy: 0.8236 - val_loss: 0.4120 - val_accuracy: 0.8516
Epoch 2/39
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3698 - accuracy: 0.8656 - val_loss: 0.3693 - val_accuracy: 0.8704
Epoch 3/39
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3311 - accuracy: 0.8781 - val_loss: 0.3459 - val_accuracy: 0.8781
Epoch 4/39
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3058 - accuracy: 0.8869 - val_loss: 0.3394 - val_accuracy: 0.8801
Epoch 5/39
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2870 - accuracy: 0.8940 - val_loss: 0.3318 - val_accuracy: 0.8827
Epoch 6/39
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2747 - accuracy: 0.8981 - val_loss: 0.3238 - val_accuracy: 0.8832
Epoch 7/39
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2573 - accuracy: 0.9045 - val_loss: 0.3314 - val_accuracy:

In [12]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.5300 - accuracy: 0.8897
[test loss, test accuracy]: [0.5299750566482544, 0.8896999955177307]
